In [1]:
import torch
import pandas as pd
from diffroute import get_node_idxs

from src.data import Dataset
from src.model import init_model
from src.helpers import training_loop, hydro_metrics, extract_train, extract_test

### Parameters

In [2]:
N = 10             # Repeat experiment 10 times
lr = .005          # Initial Runoff Learning rate
wd = .001          # LSTM Weight decay
n_epoch = 400      # training epoch
n_iter = 40        # Iteration per epoch
init_window = 100  # LSTM init window

dataset = "40_P"  # Dataset: Only 40_P, all scales give similar results.
device = "cuda:7" # Device: Assume available to NVIDIA GPU
irf_fns = [ "hayami", 'pure_lag', 'linear_storage', 'nash_cascade', 'muskingum']

### Training

In [ ]:
ds = Dataset.load_from_folder(dataset, device)
all_results = {}

for irf_fn in irf_fns:
    print(irf_fn)
    res = []
    for i in range(N):
        # Init model and optimizer
        model = init_model(ds.g, 
                           inp_size=ds.X.shape[-1], 
                           dt = .25, 
                           irf_fn=irf_fn, 
                           device=device)
        opt = torch.optim.Adam([
            {'params': model.runoff_model.parameters(), 'lr': lr, 'weight_decay': wd},
            {'params': [model.routing_params], 'lr': 1e-2, 'weight_decay': 0.0}
        ])
        scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=100, gamma=0.3)

        # Train
        losses = training_loop(ds, model, opt, n_iter=n_iter, 
                               n_epoch=n_epoch, 
                               init_window=init_window,
                               scheduler=scheduler,
                               clip_grad_norm=1.0)

        # Extract and record results
        ytr, otr = extract_train(model, ds, init_window)
        yte, ote = extract_test(model, ds, init_window)
        res.append((model.cpu(), ytr, otr, yte, ote))
        
    all_results[irf_fn]=res

hayami


 10%|█         | 4/40 [00:00<00:07,  4.54it/s]

### Evaluation

In [ ]:
yte = {k: [x[3] for x in v] for k,v in all_results.items()}
ote = {k: [x[4] for x in v] for k,v in all_results.items()}
results = {k:pd.DataFrame([hydro_metrics(out, y) \
                           for out,y in zip(ote[k], yte[k])])
           for k in ote}

In [ ]:
{k:v.std() for k,v in results.items()}

In [ ]:
{k:v.mean() for k,v in results.items()}